In [ ]:
import sys
import json
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt

sys.path.append('/code_location/multi_gpu/')
import functions_brain_network

import arch_generate_random_CNN
import importlib
importlib.reload(arch_generate_random_CNN)


In [ ]:
success = False

while not success:
    try:
        brain_net_architecture, repeating_cnn_elements = arch_generate_random_CNN.get_random_cnn_architecture()

        n_classes_dict = {'f0_label':700}

        tf.reset_default_graph()
        input_tensor = tf.placeholder(tf.float32, shape=[None, 100, 1000, 1], name='input_nervegrams')
        output_tensor, nets = functions_brain_network.make_brain_net(input_tensor,
                                                                     n_classes_dict,
                                                                     brain_net_architecture,
                                                                     trainable=True,
                                                                     batchnorm_flag=True,
                                                                     dropout_flag=True,
                                                                     save_arch_path=None,
                                                                     save_pckl_path=None,
                                                                     only_include_layers=None)
        success = True
    except ValueError:
        print('REPEAT')

print('----> INPUT:', input_tensor.shape)
for layer_dict in brain_net_architecture:
    key = layer_dict['args']['name']
    if 'conv' in key:
        print('------ {} ------'.format(key))
    if 'pool' in key or 'conv' in key:
        print(layer_dict['args'])
    print(key, nets[key].shape)

    

print('----> OUTPUT:', output_tensor.shape)

In [ ]:
def get_possible_arch(n_classes_dict={'f0_label':700}):
    success = False
    count = 0
    while not success:
#         new_net = arch_generate_random_CNN.RandomCNN()
        try:
#             brain_net_architecture = new_net.all_layer_list
            brain_net_architecture, repeating_cnn_elements = arch_generate_random_CNN.get_random_cnn_architecture()
            tf.reset_default_graph()
            input_tensor = tf.placeholder(tf.float32, shape=[None, 100, 1000, 1], name='input_nervegrams')
            output_tensor, nets = functions_brain_network.make_brain_net(input_tensor,
                                                                         n_classes_dict,
                                                                         brain_net_architecture,
                                                                         trainable=True,
                                                                         batchnorm_flag=True,
                                                                         dropout_flag=True,
                                                                         save_arch_path=None,
                                                                         save_pckl_path=None,
                                                                         only_include_layers=None)
            success = True
        except ValueError:
            count += 1
#             print(count, len(new_net.num_conv_kernels))
            pass
    return brain_net_architecture, repeating_cnn_elements['conv_layer_count'], count, nets


n_list = []
arch_list = []
nets_list = []
count = 0
for _ in range(100):
    arch, n, count_i, nets = get_possible_arch()
    n_list.append(n)
    arch_list.append(arch)
    nets_list.append(nets)
    count += count_i
    print(_, count)

In [ ]:
plt.figure()
plt.hist(n_list, 8)
plt.show()

In [ ]:
IDX = np.random.randint(len(arch_list))
print(IDX)
arch = arch_list[IDX]
nets = nets_list[IDX]
for layer_dict in arch:
    layer_name = layer_dict['args']['name']
    if 'conv' in layer_name:
        print('------ {} ------'.format(layer_name))
        print(nets[layer_name].shape)


In [ ]:
import sys
import os
import json
import glob
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

valid_format = '/om/scratch/Fri/msaddler/pitchnet/saved_models/arch_search_v00/arch_{:04d}/validation_metrics.json'
brain_format = '/om/scratch/Fri/msaddler/pitchnet/saved_models/arch_search_v00/arch_{:04d}/brain_arch.json'


def get_valid_trace(valid_metrics_fn, metric_key='f0_label:accuracy', checkpoint_number_key='step'):
    '''
    '''
    with open(valid_metrics_fn) as f: valid_metrics_dict = json.load(f)
    metric_values = valid_metrics_dict[metric_key]
    valid_step = valid_metrics_dict[checkpoint_number_key]
    ### START: WORK AROUND FOR BUG CAUSED BY PREEMPTING AND RESTARTING TRAINING (valid step is reset)
    checkpoint_numbers = [valid_step[0]]
    for idx, diff in enumerate(np.diff(valid_step)):
        if diff > 0: checkpoint_numbers.append(checkpoint_numbers[-1] + diff)
        else: checkpoint_numbers.append(checkpoint_numbers[-1] + valid_step[idx+1])
    assert len(checkpoint_numbers) == len(valid_step)
    assert len(checkpoint_numbers) == len(metric_values)
    ### END: WORK AROUND FOR BUG CAUSED BY PREEMPTING AND RESTARTING TRAINING (valid step is reset)
    return checkpoint_numbers, metric_values


def calc_num_layers(brain_arch_fn):
    with open(brain_arch_fn) as f: brain_arch = json.load(f)
    num_conv_layers = 0
    for layer_dict in brain_arch:
        if layer_dict['layer_type'] == 'tf.layers.conv2d':
            num_conv_layers = num_conv_layers + 1
    return num_conv_layers


def calc_best_metric(valid_metrics_fn, metric_key='f0_label:accuracy', maximize=True):
    '''
    '''
    with open(valid_metrics_fn) as f: valid_metrics_dict = json.load(f)
    metric_values = valid_metrics_dict[metric_key]
    if maximize: best_metric_value = np.max(metric_values)
    else: best_metric_value = np.min(metric_values)
    return best_metric_value


list_traces = []
list_num_layers = []
list_best_metric = []
list_arch_nums = []

for idx in range(750):
    brain_arch_fn = brain_format.format(idx)
    valid_metrics_fn = valid_format.format(idx)
    
    if os.path.exists(brain_arch_fn) and os.path.exists(valid_metrics_fn):
        list_traces.append(get_valid_trace(valid_metrics_fn))
        list_num_layers.append(calc_num_layers(brain_arch_fn))
        list_best_metric.append(calc_best_metric(valid_metrics_fn))
        list_arch_nums.append(idx)
        

In [ ]:
NCOLS = 5
NROWS = 1
fig, ax = plt.subplots(nrows=NROWS, ncols=NCOLS, figsize=(4*NCOLS, 4*NROWS))
ax = ax.flatten()

ax[0].hist(list_num_layers, 8, color='k')
ax[0].set_xlabel('Number of conv layers')
ax[0].set_ylabel('Count')
ax[1].hist(list_best_metric, 100, color='k')
ax[1].set_xlabel('Validation accuracy')
ax[1].set_ylabel('Count')
ax[2].plot(list_num_layers, list_best_metric, 'k.')
ax[2].set_xlabel('Number of conv layers')
ax[2].set_ylabel('Validation accuracy')
ax[3].plot(np.sort(list_best_metric), 'k.')
ax[3].set_xlabel('Arch number')
ax[3].set_ylabel('Validation accuracy')

for idx, (checkpoint_numbers, metric_values) in enumerate(list_traces):
    if list_num_layers[idx] > 0:
        ax[4].plot(checkpoint_numbers, metric_values, ls='-', lw=0.25, color='k')
    else:
        ax[4].plot(checkpoint_numbers, metric_values, ls='-', lw=0.25, color='b')
ax[4].set_xlabel('Step number')
ax[4].set_ylabel('Validation accuracy')

plt.tight_layout()
plt.show()

# save_dir = '/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_08_28_archSearch00_RSBpsychophysics/'
# fig.savefig(os.path.join(save_dir, 'arch_search_v00_summary.pdf'), bbox_inches='tight')


In [ ]:
metric_sort_idx = np.argsort(list_best_metric)
bidx = metric_sort_idx[-10]
print(list_arch_nums[bidx], list_best_metric[bidx], list_num_layers[bidx])

In [ ]:
import os
import json

output_dir = '/om2/user/msaddler/pitchnet/saved_models/PND_v04_TLAS_classification0'
arch_fn = os.path.join(output_dir, 'brain_arch.json')

arch_fn_list = [
'/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0703/brain_arch.json',
'/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0693/brain_arch.json',
'/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0156/brain_arch.json',
'/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0051/brain_arch.json',
'/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0342/brain_arch.json',
'/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0664/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0084/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0265/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0637/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0086/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0254/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0416/brain_arch.json',
]

# arch_fn_list = [
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0628/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0374/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0471/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0018/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0610/brain_arch.json',
# '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0429/brain_arch.json',
# # '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0721/brain_arch.json',
# # '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0282/brain_arch.json',
# # '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0682/brain_arch.json',
# # '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0533/brain_arch.json',
# # '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0466/brain_arch.json',
# # '/om/scratch/Sun/msaddler/pitchnet/saved_models/arch_search_v00/arch_0284/brain_arch.json',
# ]

for arch_fn in arch_fn_list:

    with open(arch_fn) as f:
        arch_list = json.load(f)
    print('_' * len(arch_fn))
    print(arch_fn)
    for layer in arch_list:
        if layer['layer_type'] == 'tf.layers.conv2d':
            print('layer: {} | kernel_shape: {} | filters: {}'.format(
                layer['args']['name'], layer['args']['kernel_size'], layer['args']['filters']))
        elif 'pool' in layer['layer_type']:
            print('layer: {} | strides: {} | pool_size: {}'.format(
                layer['args']['name'], layer['args']['strides'], layer['args']['pool_size']))
        elif layer['layer_type'] == 'tf.layers.dense':
            print('layer: {} | units: {}'.format(
                layer['args']['name'], layer['args']['units']))
        elif layer['layer_type'] in ['tf.nn.relu',
                                     'tf.layers.batch_normalization',
                                     'tf.layers.dropout',
                                     'fc_top_classification'
                                    ]:
            print('layer: {}'.format(layer['args']['name']))

In [ ]:
import arch_generate_random_CNN
import importlib
importlib.reload(arch_generate_random_CNN)

for _ in range(20):
    layer_list, repeating_cnn_elements = arch_generate_random_CNN.get_random_cnn_architecture()
#         kwargs_sample_repeating_cnn_elements={'max_kernel_area':100})
    conv_layer_count = repeating_cnn_elements['conv_layer_count']
    print('----')
    for layer_index in range(conv_layer_count):
        kernel_shapes = repeating_cnn_elements['conv_kernel_shapes'][layer_index]
        kernel_depths = repeating_cnn_elements['conv_kernel_depths'][layer_index]
        pool_strides = repeating_cnn_elements['pool_strides'][layer_index]
        print(layer_index, kernel_shapes+[kernel_depths], pool_strides)
    